# PAOS

This notebook demonstrates the basic functionality of PAOS.

Select the desired example configuration desired below

In [1]:
import os, time
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import paos

## Inputs

Define lens file to use

In [2]:
OpticalPrescription = 'example.ini'
# OpticalPrescription = 'lens_file_TA_Ground.ini'

OpticalPrescriptionFile = os.path.join('../lens data', OpticalPrescription)

# Parse lens file
from paos.core.parseConfig import parse_config
pup_diameter, parameters, wavelengths, fields, opt_chains = parse_config(OpticalPrescriptionFile)

## Ray tracing
Implement a full raytracing. For diagnostic purposes.

In [3]:
paos.raytrace(fields[0], opt_chains[0], y=-0.0)

['S02 - M1              y:  0.000mm ut: 0.000e+00 rad x:  0.000mm us: 0.000e+00 rad',
 'S03 - Differential magnification y:  0.000mm ut: 0.000e+00 rad x:  0.000mm us: 0.000e+00 rad',
 'S04 - IMAGE_PLANE     y:  0.000mm ut: 0.000e+00 rad x:  0.000mm us: 0.000e+00 rad']

In [4]:
start_time = time.time()
print('Start POP...')

retval = paos.run(pup_diameter, 1.0e-6 * wavelengths[0], parameters['grid_size'], parameters['zoom'], fields[0], opt_chains[0])

end_time = time.time()
print('POP completed in {:6.1f}s'.format(end_time-start_time))

Start POP...
POP completed in    0.2s


## Plot results

In [5]:
plots_dir = './plots'
Path(plots_dir).mkdir(parents=True, exist_ok=True)
figname = os.path.join(plots_dir, ''.join([OpticalPrescription, '_', str(1.0e-6 * wavelengths[0]), '.png']))
paos.plot_pop(retval, ima_scale='log', ncols=3, figname=figname)#, surface_zoom={23:5})

In [7]:
retval[3]['ABCDt'].f_eff, retval[3]['ABCDt'].f_eff/(5/6), retval[3]['fratio']

(50.0, 60.0, 59.99999949576094)

In [12]:
0.5 * 1.25

0.625

# Save results

This section demonstrates saving your results.
The results are saved in .h5 format.

The saving routine are very flexible: you can either
- save everything in your results (wfo, dx, dx, apertures...)
- or just a selection of things that you need (just the wfo, or whatever list of things)

In [7]:
saving_dir = './data'
Path(saving_dir).mkdir(parents=True, exist_ok=True)

1) This is how to save your output if you simulated just one POP realisation. 

In [8]:
# HDF5FileName = os.path.join(saving_dir, ''.join([OpticalPrescription, '_everything.h5']))
# paos.save_output(retval, HDF5FileName, overwrite=True)

HDF5FileName = os.path.join(saving_dir, ''.join([OpticalPrescription, '_selection.h5']))
paos.save_output(retval, HDF5FileName, keys_to_keep=['wfo', 'dx'], overwrite=True)

paos - INFO - saving ./data/Ariel_AIRS-CH1.ini_selection.h5 started...
paos - INFO - removing old file
paos - INFO - saving ended.


2) This is how to save your output if you simulated more than one POP realisation. 

In [9]:
# Since we just ran the POP at two different wavelengths, 
# let's use them to unequivocally tag the two simulations.

group_tags = list(map(str, wavelengths))

In [10]:
# HDF5FileName = os.path.join(saving_dir, ''.join([OpticalPrescription, '_everything_cube.h5']))
# paos.save_datacube(ret, HDF5FileName, group_tags, overwrite=True)

HDF5FileName = os.path.join(saving_dir, ''.join([OpticalPrescription, '_selection_cube.h5']))
paos.save_datacube(ret, HDF5FileName, group_tags,
                   keys_to_keep=['amplitude', 'dx', 'dy'], overwrite=True)

paos - INFO - Saving ./data/Ariel_AIRS-CH1.ini_selection_cube.h5 started...
paos - INFO - Remove old file
paos - INFO - Saving ended.
